In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from difflib import get_close_matches, SequenceMatcher
from sklearn.metrics import mutual_info_score
from sklearn.metrics import precision_score, recall_score, f1_score
from random import choice, randrange
import requests
import time
import string
import random
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import json
import nltk
nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import words
from nltk.corpus import stopwords
!pip install wikipedia
import wikipedia
!pip install pyinterval
import os
import tarfile
import time
import pickle

[nltk_data] Downloading package words to
[nltk_data]     /Users/enzoveltri/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/enzoveltri/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/enzoveltri/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
## data prep
ps = PorterStemmer()
stemmed_words_list = [ps.stem(w) for w in words.words()]

In [3]:
## utils funct
def saveCache(fileName, cache):
    tempDict = {}
    for key, value in cache.items():
        tempDict[key] = list(value)
    with open(fileName, 'w') as fp:
        json.dump(tempDict, fp)
        
def loadCache(fileName):
    data = None
    with open(fileName, 'r') as fp:
        data = json.load(fp)
    cache = {}
    for key, value in data.items():
        cache[key] = set(value)
    return cache

def findFromCached(cachedAlias, cachedProvenance, columns):
    aliasReturn = {}
    provenanceReturn = {}
    columnsToSearch = set()
    for column in columns:
        if column in cachedAlias:
            aliasReturn[column] = cachedAlias[column]
        else:
            columnsToSearch.add(column)
        if column in cachedProvenance:
            provenanceReturn[column] = cachedProvenance[column]
        else:
            columnsToSearch.add(column)
     
    return aliasReturn, provenanceReturn, columnsToSearch

In [4]:
## caches
cacheSynonym = {}
cacheRelatedTo = {}
cacheIsA = {}
cacheDerivedFrom = {}
cacheWikipedia = {}
cachedAlias = {}
cachedProvenance = {}

In [5]:
## load caches
cacheSynonym = loadCache('./cacheSynonym-small.json')
cacheRelatedTo = loadCache('./cacheRelatedTo-small.json')
cacheIsA = loadCache('./cacheIsA-small.json')
cacheDerivedFrom = loadCache('./cacheDerivedFrom-small.json')
cacheWikipedia = loadCache('./cacheWikipedia-small.json')
cachedAlias = loadCache('./cacheAlias.json')
#cachedProvenance = loadCache('./cacheProvenance.json')

In [6]:
a_file = open("cacheProvenance.pkl", "rb")
cachedProvenance = pickle.load(a_file)

In [7]:
def getLCS(string1, string2):
    match = SequenceMatcher(None, string1, string2).find_longest_match(0, len(string1), 0, len(string2))
    return string1[match.a: match.a + match.size]
    #return string2[match.b: match.b + match.size]

## CONCEPTNET.IO ##
def getSynonym(word, limit=10, dropWord=True, useCache=True):
    processedText = word.lower().replace(' ', '_')
    synonyms = set()
    if (len(processedText) <= 1):
        return synonyms, True
    if useCache and word in cacheSynonym:
        return cacheSynonym[word], True
    baseUrl = "https://api.conceptnet.io/query?node=/c/en/$WORD$&other=/c/en&limit=$LIMIT$&rel=/r/Synonym"
    processedURL = baseUrl.replace("$WORD$", processedText)
    processedURL = processedURL.replace("$LIMIT$", str(limit))
    #print("Processed URL: ", processedURL)
    obj = None
    try:
        obj = requests.get(processedURL).json()
    except Exception:
        return synonyms, False
    #obj = requests.get(processedURL).json()
    if 'edges' not in obj:
        return synonyms, False
    resultsLen = len(obj['edges'])
    edges = obj['edges']
    #print("Results:", resultsLen)
    
    for i in range(0, resultsLen):
        edge = edges[i]
        label = edge['end']['label']
        synonyms.add(label.lower())
        label = edge['start']['label']
        synonyms.add(label.lower())
    if dropWord:
        synonyms.discard(word.lower())
    if useCache:
        cacheSynonym[word] = synonyms
    return synonyms, False

def getRelatedTo(word, limit=10, dropWord=True, useCache=True):
    processedText = word.lower().replace(' ', '_')
    relatedTo = set()
    if (len(processedText) <= 1):
        return relatedTo, True
    if useCache and word in cacheRelatedTo:
        return cacheRelatedTo[word], True
    baseUrl = "https://api.conceptnet.io/query?node=/c/en/$WORD$&other=/c/en&limit=$LIMIT$&rel=/r/RelatedTo"
    processedURL = baseUrl.replace("$WORD$", processedText)
    processedURL = processedURL.replace("$LIMIT$", str(limit))
    #print("Processed URL: ", processedURL)
    obj = None
    try:
        obj = requests.get(processedURL).json()
    except Exception:
        return relatedTo, False
    #obj = requests.get(processedURL).json()
    if 'edges' not in obj:
        return relatedTo, False
    resultsLen = len(obj['edges'])
    edges = obj['edges']
    #print("Results:", resultsLen)
    for i in range(0, resultsLen):
        edge = edges[i]
        label = edge['end']['label']
        relatedTo.add(label.lower())
        label = edge['start']['label']
        relatedTo.add(label.lower())
    if dropWord:
        relatedTo.discard(word.lower())
    if useCache:
        cacheRelatedTo[word] = relatedTo
    return relatedTo, False

def getIsA(word, limit=10, dropWord=True, useCache=True):
    processedText = word.lower().replace(' ', '_')
    isA = set()
    if (len(processedText) <= 1):
        return isA, True
    if useCache and word in cacheIsA:
        return cacheIsA[word], True
    baseUrl = "https://api.conceptnet.io/query?node=/c/en/$WORD$&other=/c/en&limit=$LIMIT$&rel=/r/IsA"
    processedURL = baseUrl.replace("$WORD$", processedText)
    processedURL = processedURL.replace("$LIMIT$", str(limit))
    #print("Processed URL: ", processedURL)
    obj = None
    try:
        obj = requests.get(processedURL).json()
    except Exception:
        return isA, False
    #obj = requests.get(processedURL).json()
    if 'edges' not in obj:
        return isA, False
    resultsLen = len(obj['edges'])
    edges = obj['edges']
    #print("Results:", resultsLen)
    for i in range(0, resultsLen):
        edge = edges[i]
        label = edge['end']['label']
        isA.add(label.lower())
    if dropWord:
        isA.discard(word.lower())
    if useCache:
        cacheIsA[word] = isA
    return isA, False

def getDerivedFrom(word, limit=10, dropWord=True, useCache=True):
    processedText = word.lower().replace(' ', '_')
    derivedFrom = set()
    if (len(processedText) <= 1):
        return derivedFrom, True
    if useCache and word in cacheDerivedFrom:
        return cacheDerivedFrom[word], True
    baseUrl = "https://api.conceptnet.io/query?node=/c/en/$WORD$&other=/c/en&limit=$LIMIT$&rel=/r/DerivedFrom"
    processedURL = baseUrl.replace("$WORD$", processedText)
    processedURL = processedURL.replace("$LIMIT$", str(limit))
    #print("Processed URL: ", processedURL)
    obj = None
    try:
        obj = requests.get(processedURL).json()
    except Exception:
        return derivedFrom, False
    #obj = requests.get(processedURL).json()
    if 'edges' not in obj:
        return derivedFrom, False
    resultsLen = len(obj['edges'])
    edges = obj['edges']
    #print("Results:", resultsLen)
    for i in range(0, resultsLen):
        edge = edges[i]
        label = edge['end']['label']
        if (label != word):
            derivedFrom.add(label.lower())
    if dropWord:
        derivedFrom.discard(word.lower())
    if useCache:
        cacheDerivedFrom[word] = derivedFrom
    return derivedFrom, False

## LCS ##
def getAmbiguousWithLCS(useStemming, col1, columns):
    translate_table = dict((ord(char), None) for char in string.punctuation)
    ambiguousValues = set()     
    for col2 in columns:
        if (col1 != col2):
            lcs = getLCS(col1, col2)
            minLen = min(len(col1), len(col2))
            maxLen = max(len(col1), len(col2))
            if (2 * minLen) < maxLen:
                continue
            if (len(lcs) >= (0.5*minLen) and len(lcs) > 1):
                lcs = lcs.translate(translate_table)
                if (lcs.lower() in stopwords.words('english')):
                    continue
                if (useStemming):
                    words_in_lcs = word_tokenize(lcs)
                    lcs_stemmed = ""
                    for w in words_in_lcs:
                        stemmedW = ps.stem(w)
                        if (stemmedW in stemmed_words_list):
                            lcs_stemmed += stemmedW + " "
                    if (len(lcs_stemmed) > 0):
                        #print(col1, col2, lcs, lcs_stemmed, sep="\t")
                        ambiguousValues.add(lcs)
                else:
                    #print(col1, col2, lcs, sep="\t")
                    ambiguousValues.add(lcs)
    return ambiguousValues

## WIKIPEDIA ##
def getAmbiguityFromWikipedia(column, useStemming=True, results=2, useCache=True):
        if useCache and column in cacheWikipedia:
            return cacheWikipedia[column], True
        wikipediaResults = set()
        try:
            wikipediaResults = set(wikipedia.search(column.replace("-"," "), results))
            wikipediaResultsStrip = set()
            for result in wikipediaResults:
                wikipediaResultsStrip.add(result.lower().strip())
            wikipediaResults = wikipediaResultsStrip
        except Exception:
                pass
        if (useStemming):
            translate_table = dict((ord(char), None) for char in string.punctuation)
            columnNoPunct = column.translate(translate_table)
            stemmed_columns = [ps.stem(w) for w in word_tokenize(columnNoPunct)]
            wikipediaResultsStemmed = set()
            for wikipediaResult in wikipediaResults:
                wikipediaResult = wikipediaResult.translate(translate_table)
                words_in_wiki = word_tokenize(wikipediaResult)
                stemmed_wiki = ""
                for w in words_in_wiki:
                    w_stemmed = ps.stem(w)
                    if w_stemmed in stemmed_columns:
                        stemmed_wiki = w + " "
                if (len(stemmed_wiki) > 0):
                    wikipediaResultsStemmed.add(stemmed_wiki.lower().strip())
            if useCache:
                cacheWikipedia[column] = wikipediaResultsStemmed
            return wikipediaResultsStemmed, True
        else:
            if useCache:
                cacheWikipedia[column] = wikipediaResults
            return wikipediaResults, False

## FUNCTIONS ##

def getLabel(columns, limit=10, useStemming=True, useLCS=False):
    labels = {}
    provenance = {}
    for column in columns:
        provenanceMap = {}
        #start_time = time.time()
        synonyms, cachedSynonyms = getSynonym(column.replace("-"," "), limit=limit, dropWord=True, useCache=True)
        #print("Synonyms time: %s" %(time.time() - start_time))
        #start_time = time.time()
        relatedTo, cachedrelatedTo = getRelatedTo(column.replace("-"," "), limit=limit, dropWord=True, useCache=True)
        #print("RelatedTo time: %s" %(time.time() - start_time))
        #start_time = time.time()
        isA, cachedIsA = getIsA(column.replace("-"," "), limit=limit, dropWord=True, useCache=True)
        #print("IsA time: %s" %(time.time() - start_time))
        #start_time = time.time()
        derivedFrom, cachedDerivedFrom = getDerivedFrom(column.replace("-"," "), limit=limit, dropWord=True, useCache=True)
        #print("DerivedFrom time: %s" %(time.time() - start_time))
        #start_time = time.time()
        #wikipediaResults = set(wikipedia.search(column.replace("-"," "), results=2))
        wikipediaResults, cachedWikipediaResults = getAmbiguityFromWikipedia(column.replace("-"," "), useStemming=True, results=2, useCache=True)
        #print("Wikipedia time: %s" %(time.time() - start_time))
        #start_time = time.time()
        lcsAmb = getAmbiguousWithLCS(useStemming, column, columns)
        #print("LSC time: %s" %(time.time() - start_time))
        labelsForColumn = set()
        if len(synonyms) > 0:
            labelsForColumn = labelsForColumn | set(synonyms)
        if len(relatedTo) > 0:
            labelsForColumn = labelsForColumn | set(relatedTo)
        if len(wikipediaResults) > 0:
            labelsForColumn = labelsForColumn | set(wikipediaResults)
        if len(isA) > 0:
            labelsForColumn = labelsForColumn | set(isA)
        if len(derivedFrom) > 0:
            labelsForColumn = labelsForColumn | set(derivedFrom)
        #labelsForColumn = synonyms | relatedTo | wikipediaResults | isA | derivedFrom
        if len(labelsForColumn) == 0:
            labelsForColumn = labelsForColumn | lcsAmb
        if useLCS:
            labelsForColumn = labelsForColumn | lcsAmb
        provenanceMap['synonyms'] = synonyms
        provenanceMap['relatedTo'] = relatedTo
        provenanceMap['isA'] = isA
        provenanceMap['derivedFrom'] = derivedFrom
        provenanceMap['wikipediaResults'] = wikipediaResults
        provenanceMap['lcsAmb'] = lcsAmb
        labelsForColumn = [w.lower().strip() for w in list(labelsForColumn)]
        labels[column] = set(labelsForColumn)
        provenance[column] = provenanceMap
        if not (cachedSynonyms or cachedrelatedTo or cachedIsA or cachedDerivedFrom):
            time.sleep(1)
    return labels, provenance

def getAmbiguousColumns(columns, aliasDict):
    colSet = set()
    ambiguousLabels = {}
    ambiguousAttr = {}
    for col1 in columns:
        for col2 in columns:
            if col1 != col2:
                alias1 = aliasDict[col1]
                alias2 = aliasDict[col2]
                ambiguousValues = set(alias1).intersection(set(alias2))
                if (len(ambiguousValues) > 0):
                    colSet.add(col1)
                    colSet.add(col2)
                    if col1 not in ambiguousLabels:
                        ambiguousLabels[col1] = set(ambiguousValues)
                    else:
                        setValue = ambiguousLabels[col1]
                        setValue = setValue | ambiguousValues
                        ambiguousLabels[col1] = setValue
                    if col2 not in ambiguousLabels:
                        ambiguousLabels[col2] = set(ambiguousValues)
                    else:
                        setValue = ambiguousLabels[col2]
                        setValue = setValue | ambiguousValues
                        ambiguousLabels[col2] = setValue
                    if col1 not in ambiguousAttr:
                        setAttr = set()
                        ambiguousAttr[col1] = setAttr
                    setAttr = ambiguousAttr[col1]
                    setAttr.add(col2)
    return colSet, ambiguousLabels, ambiguousAttr

def updateAlias(dic, dicAdd, dicRemove, blackList = set()):
    for key, value in dicAdd.items():
        if key not in dic:
            dic[key] = set(value)
        else:
            setValue = dic[key]
            for v in value:
                setValue.add(v)
    for key, value in dicRemove.items():
        if key in dic:
            setValue = dic[key]
            for v in value:
                setValue.discard(v)
    #print("BLACKLIST:")
    #print(blackList)
    for key, value in dic.items():
        #print("BEFORE:")
        #print(value)
        value = value - blackList
        dic[key] = value
        #print("AFTER:")
        #print(value)
    return dic


In [8]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

def getStatsForValues(df, columns): 
    numerical = {}
    categorical = {}
    text = {}
    for column in columns:
        df[column] = pd.to_numeric(df[column], errors='ignore')
        columnType = df[column].dtype
        if (columnType in numerics):
            stats = df[column].describe()
            min = stats['min']
            max = stats['max']
            mean = stats['mean']
            median = stats['50%']
            std = stats['std']
            stats = (min, max, mean, median, std)
            numerical[column] = stats
        else:
            lenValue = df[column].nunique()
            lenColumn = df[column].shape[0]
            threshold = lenColumn * 0.9
            if (lenValue < threshold):
                #compute frequencies
                freq = df[column].value_counts()
                categorical[column] = freq
            else:
                mean = df[column].map(len).mean()
                std = df[column].map(len).std()
                median = df[column].map(len).median()
                tmpCol = df[column].sort_values()
                min = tmpCol.iloc[0]
                max = tmpCol.iloc[-1]
                stats = (min, max, mean, median, std)
                numerical[column] = stats

    return numerical, categorical, text

In [9]:
def findAmbiguousForCategorical(categorical):
    colSet = set()
    ambiguousValuesForAttr = {}
    setPairs = {}
    for c1, freq1 in categorical.items():
        for c2, freq2 in categorical.items():
            if (c1 != c2):
                keys1 = freq1.keys()
                keys2 = freq2.keys()
                ambiguousValues = set(keys1).intersection(set(keys2))
                if (len(ambiguousValues) < 1):
                    continue
                colSet.add(c1)
                colSet.add(c2)
                if c1 not in ambiguousValuesForAttr:
                    ambiguousValuesForAttr[c1] = set(ambiguousValues)
                else:
                    setValue = ambiguousValuesForAttr[c1]
                    setValue = setValue | ambiguousValues
                    ambiguousValuesForAttr[c1] = setValue
                if c1 not in setPairs:
                    attrs = set()
                    attrs.add(c2)
                    setPairs[c1] = attrs
                else:
                    attrs = setPairs[c1]
                    attrs.add(c2)
    return colSet, ambiguousValuesForAttr, setPairs

def getIntersection(min1, max1, min2, max2):
    ## check type
    if (type(min1) == str) or (type(max1) == str) or (type(min2) == str) or (type(max2) == str):
        return None
    ## check intervals
    if (max1 < min1) or (max2 < min2):
        print("Error min1-max1: {}-{} min2-max2:{}-{}".format(min1, max1, min2, max2))
        return None
    # case min1 - max1, min2 - max2 --> no intersection
    if (max1 < min2):
        return None
    # case min2 - max2, min1 - max2 --> no intersection
    if (max2 < min1):
        return None
    # case min1, min2, max1, max2 --> min2-max1
    if (min1 <= min2) and (min2 <= max1) and (max1 <= max2):
        return (min2, max1)
    # case min2, min1, max2, max1 --> min2-max1
    if (min2 <= min1) and (min1 <= max2) and (max2 <= max1):
        return (min1, max2)
    # case min1, min2, max2, max1 --> min2-max2
    if (min1 <= min2) and (min2 <= max2) and (max2 <= max1):
        return (min2, max2)
    # case min2, min1, max1, max2 --> min1-max1
    return (min1, max1)


def findAmbiguousForNumerical(numerical):
    ## naive strategy: intersection of intervals
    colSet = set()
    ambiguousValuesForAttr = {}
    setPairs = {}
    for c1, stats1 in numerical.items():
        for c2, stats2 in numerical.items():
            if (c1 != c2):
                interval = getIntersection(stats1[0], stats1[1], stats2[0], stats2[1])
                if (interval is None):
                    continue
                colSet.add(c1)
                colSet.add(c2)
                if c1 not in ambiguousValuesForAttr:
                    listValue = []
                    listValue.append(interval)
                    ambiguousValuesForAttr[c1] = listValue
                else:
                    listValue = ambiguousValuesForAttr[c1]
                    listValue.append(interval)
                if c1 not in setPairs:
                    attrs = set()
                    attrs.add(c2)
                    setPairs[c1] = attrs
                else:
                    attrs = setPairs[c1]
                    attrs.add(c2)
    return colSet, ambiguousValuesForAttr, setPairs

In [10]:
def findAmbiguousForLabelAndData(setPairs, setPairsCategorical, setPairsNumerical, ambiguousValuesForAttrCategorical, ambiguousValuesForAttrNumerical):
    setPairsLabelData = {}
    ambiguousValues = {}
    for attributeLabel, ambiguousAttributesLabel in setPairs.items():
        setAmb = set()
        for attributeCategorical, ambiguousAttributesCategorical in setPairsCategorical.items():
            if (attributeLabel == attributeCategorical):
                intersection = ambiguousAttributesLabel.intersection(ambiguousAttributesCategorical)
                #intersection = list(intersection)
                if (len(intersection) > 0):
                    for amb in intersection:
                        setAmb.add(amb)
                    for attr in intersection:
                        valuesAmb = ambiguousValuesForAttrCategorical[attr]
                        for value in valuesAmb:
                            if value in ambiguousValues:
                                setAttrs = ambiguousValues[value]
                                setAttrs.add(attr)
                            else:
                                setAttrs = set()
                                setAttrs.add(attr)
                                ambiguousValues[value] = setAttrs
        for attributeNumerical, ambiguousAttributesNumerical in setPairsNumerical.items():
            if (attributeLabel == attributeNumerical):
                intersection = ambiguousAttributesLabel.intersection(ambiguousAttributesNumerical)
                #intersection = list(intersection)
                if (len(intersection) > 0):
                    for amb in intersection:
                        setAmb.add(amb)
                    print(intersection)
                    for attr in intersection:
                        intervals = ambiguousValuesForAttrNumerical[attr]
                        for interval in intervals:
                            minI = interval[0]
                            maxI = interval[1]
                            if minI in ambiguousValues:
                                setAttrs = ambiguousValues[minI]
                                setAttrs.add(attr)
                            else:
                                setAttrs = set()
                                setAttrs.add(attr)
                                ambiguousValues[minI] = setAttrs
                            if maxI in ambiguousValues:
                                setAttrs = ambiguousValues[maxI]
                                setAttrs.add(attr)
                            else:
                                setAttrs = set()
                                setAttrs.add(attr)
                                ambiguousValues[maxI] = setAttrs
        setPairsLabelData[attributeLabel] = setAmb
    return setPairsLabelData, ambiguousValues

def reversedMap(map):
    reversed = {}
    for attrName, values in map.items():
        for value in values:
            #print(attrName, value)
            if (value not in reversed):
                    attrSet = set()
                    attrSet.add(attrName)
                    reversed[value] = attrSet
            else:
                    attrSet = reversed[value]
                    attrSet.add(attrName)
            #if type(value) is tuple:
            #    print("TUPLE")    
    return reversedMap

# Data retrieve

In [11]:
## WDC download
## one time execution
skipDownload = True
if skipDownload == False:
    #for i in range(0, 25):
    for i in range(2, 4):    
        s = str(i)
        if i < 10:
            s = '0' + str(i)
        url = "http://data.dws.informatik.uni-mannheim.de/webtables/2015-07/englishCorpus/compressed/" + s + ".tar.gz"
        response = requests.get(url, stream=True)
        file = tarfile.open(fileobj=response.raw, mode="r|gz")
        path = "./" + s + "/"
        file.extractall(path=path)
        file.close()
        tarFile = "./"+s+"/"+s+".tar"
        !python -m tarfile -e {tarFile} ./WDC/

        #os.remove(path)
        os.remove(tarFile)
        #os.remove('./WDC/')

In [29]:
## WDC Web Table Corpus 2015
## All data
relationalTables_all = []
parsedSchemas_all = []

#paths_to_json = ["./WDC/0", './WDC/1','./WDC/2','./WDC/3','./WDC/4','./WDC/5']
paths_to_json = ["./WDC/5"]

for path_to_json in paths_to_json:
    print(path_to_json)
    #path_to_json = './WDC/0'
    #path_to_json = './WDC/1'
    #path_to_json = './WDC/2'
    #path_to_json = './WDC/5'
    
    relationalTables = []
    parsedSchemas = []
    json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

    start_time = time.time()
    for json_file in json_files:
        path = path_to_json + "/" + json_file
        with open(path) as f:
            data = json.load(f)
            if (data['tableType'] == 'RELATION') and (data['hasHeader'] == True) and (data['tableOrientation'] == "HORIZONTAL") and (data['headerPosition'] == "FIRST_ROW"):
                relationalTables.append(data)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Loaded Relational JSON Files:", len(relationalTables))
    print("Loaded in ", elapsed_time)

    #parsedSchemas = {}

    start_time = time.time()
    for table in relationalTables:
    #for table in relationalTables[:5000]:
        relationData = table['relation']
        #pageTitle = table['pageTitle'].strip()
        #title = table['title'].strip()
        #print("PageTitle:", pageTitle)
        #print("Title:",title)
        columns = []
        data = {}
        for column in relationData:
            columnName = column[0].strip()
            if columnName != '':
                columns.append(columnName)
                data[columnName] = column[1:]
        if len(data) > 0:
            t = (columns, data)
            parsedSchemas.append(t)
    print("Parsed schemas: ", len(parsedSchemas))
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Parsed in ", elapsed_time)
    relationalTables_all.extend(relationalTables)
    parsedSchemas_all.extend(parsedSchemas)
    
del relationalTables, relationalTables_all

parsedSchemas = parsedSchemas_all
print("Parsed schemas: ", len(parsedSchemas))

./WDC/5
Loaded Relational JSON Files: 896801
Loaded in  768.5963201522827
Parsed schemas:  894221
Parsed in  22.032452821731567
Parsed schemas:  894221


In [33]:
structuredData = []
#K = 1500000
#K = 1000
#K = 5000
#K = 10000
#K = 25000
#K = 50000
#K = 75000
#K = 100000
#K = 150000
#K = 300000
#K = 400000
#K = 500000
#K = 750000
#K = 1000000
#K = 1500000
K = 5000000

stats = {"tables": 0, "columns": 0, 'synonyms':0, 'relatedTo':0, 'isA':0, 'derivedFrom':0, 'wikipediaResults':0, 'lcsAmb':0}

#for table, schemaAndData in parsedSchemas.items():
#for schemaAndData in parsedSchemas:
count = 0
start_time = time.time()
for schemaAndData in parsedSchemas:
    columns = schemaAndData[0]
    #print(columns)
    aliasDict, provenance, colsToFind = findFromCached(cachedAlias, cachedProvenance, columns)
    if (len(colsToFind) > 0):
        #search online
        #print("Search online since not stored in the cache for alias")
        aliasDictOnline, provenanceOnline = getLabel(colsToFind)
        #print(aliasDictOnline)
        #print(provenanceOnline)
        aliasDict.update(aliasDictOnline)
        provenance.update(provenanceOnline)
        cachedAlias.update(aliasDict)
        cachedProvenance.update(provenance)
    #aliasDict, provenance = getLabel(columns)
    #print(aliasDict)
    #print(provenance)
    ## TODO: periodically save caches
    colSet, ambiguousLabels, setPairs = getAmbiguousColumns(columns, aliasDict)
    #print(colSet)
    #print(ambiguousLabels)
    #print(setPairs)
    #print(provenance)
    colAmbiguous = list(colSet)
    count += 1
    if (count % 100) == 0:
        print("Processed: ", count)
        print("Elapsed time: %s" %(time.time() - start_time))
        print("StructuredData size: ", len(structuredData))
    if (len(colSet) > 0):
        stats['tables'] = stats['tables'] + 1
        stats['columns'] = stats['columns'] + len(columns)
        table = schemaAndData[1]
        t = (table, columns, setPairs, ambiguousLabels, provenance)
        structuredData.append(t)
    if len(structuredData) == K:
        break
print("Process ended in time: %s" %(time.time() - start_time))
print("Structured data: ", len(structuredData))

Processed:  100
Elapsed time: 0.02394890785217285
StructuredData size:  17
Processed:  200
Elapsed time: 0.042618751525878906
StructuredData size:  41
Processed:  300
Elapsed time: 0.053404808044433594
StructuredData size:  63
Processed:  400
Elapsed time: 0.0661919116973877
StructuredData size:  94
Processed:  500
Elapsed time: 0.07577300071716309
StructuredData size:  124
Processed:  600
Elapsed time: 0.0876309871673584
StructuredData size:  151
Processed:  700
Elapsed time: 0.0987539291381836
StructuredData size:  175
Processed:  800
Elapsed time: 0.10793089866638184
StructuredData size:  206
Processed:  900
Elapsed time: 0.11787700653076172
StructuredData size:  242
Processed:  1000
Elapsed time: 0.12786602973937988
StructuredData size:  276
Processed:  1100
Elapsed time: 0.13626599311828613
StructuredData size:  305
Processed:  1200
Elapsed time: 0.14888286590576172
StructuredData size:  332
Processed:  1300
Elapsed time: 0.15846920013427734
StructuredData size:  371
Processed:  1

Processed:  11400
Elapsed time: 1.1572680473327637
StructuredData size:  3317
Processed:  11500
Elapsed time: 1.167111873626709
StructuredData size:  3346
Processed:  11600
Elapsed time: 1.1752619743347168
StructuredData size:  3375
Processed:  11700
Elapsed time: 1.183938980102539
StructuredData size:  3406
Processed:  11800
Elapsed time: 1.1976840496063232
StructuredData size:  3443
Processed:  11900
Elapsed time: 1.2097458839416504
StructuredData size:  3474
Processed:  12000
Elapsed time: 1.2244741916656494
StructuredData size:  3504
Processed:  12100
Elapsed time: 1.2342908382415771
StructuredData size:  3530
Processed:  12200
Elapsed time: 1.2416341304779053
StructuredData size:  3564
Processed:  12300
Elapsed time: 1.2506308555603027
StructuredData size:  3591
Processed:  12400
Elapsed time: 1.2593321800231934
StructuredData size:  3621
Processed:  12500
Elapsed time: 1.271655797958374
StructuredData size:  3645
Processed:  12600
Elapsed time: 1.2808640003204346
StructuredData s

Processed:  22000
Elapsed time: 2.319991111755371
StructuredData size:  6412
Processed:  22100
Elapsed time: 2.3254899978637695
StructuredData size:  6439
Processed:  22200
Elapsed time: 2.334117889404297
StructuredData size:  6473
Processed:  22300
Elapsed time: 2.3428380489349365
StructuredData size:  6500
Processed:  22400
Elapsed time: 2.3561558723449707
StructuredData size:  6534
Processed:  22500
Elapsed time: 2.3641579151153564
StructuredData size:  6559
Processed:  22600
Elapsed time: 2.3733150959014893
StructuredData size:  6594
Processed:  22700
Elapsed time: 2.3832881450653076
StructuredData size:  6626
Processed:  22800
Elapsed time: 2.392388105392456
StructuredData size:  6655
Processed:  22900
Elapsed time: 2.400988817214966
StructuredData size:  6689
Processed:  23000
Elapsed time: 2.408111095428467
StructuredData size:  6716
Processed:  23100
Elapsed time: 2.4167978763580322
StructuredData size:  6741
Processed:  23200
Elapsed time: 2.425261974334717
StructuredData size

Processed:  32700
Elapsed time: 3.4599759578704834
StructuredData size:  9514
Processed:  32800
Elapsed time: 3.4750001430511475
StructuredData size:  9542
Processed:  32900
Elapsed time: 3.4831628799438477
StructuredData size:  9567
Processed:  33000
Elapsed time: 3.494702100753784
StructuredData size:  9597
Processed:  33100
Elapsed time: 3.507755994796753
StructuredData size:  9629
Processed:  33200
Elapsed time: 3.517359733581543
StructuredData size:  9653
Processed:  33300
Elapsed time: 3.5268290042877197
StructuredData size:  9688
Processed:  33400
Elapsed time: 3.536579132080078
StructuredData size:  9709
Processed:  33500
Elapsed time: 3.5470290184020996
StructuredData size:  9740
Processed:  33600
Elapsed time: 3.556285858154297
StructuredData size:  9764
Processed:  33700
Elapsed time: 3.5673458576202393
StructuredData size:  9797
Processed:  33800
Elapsed time: 3.5762970447540283
StructuredData size:  9825
Processed:  33900
Elapsed time: 3.5901827812194824
StructuredData siz

Processed:  44500
Elapsed time: 4.878216981887817
StructuredData size:  12951
Processed:  44600
Elapsed time: 4.884966850280762
StructuredData size:  12979
Processed:  44700
Elapsed time: 4.897807836532593
StructuredData size:  13009
Processed:  44800
Elapsed time: 4.909771919250488
StructuredData size:  13038
Processed:  44900
Elapsed time: 4.921234846115112
StructuredData size:  13069
Processed:  45000
Elapsed time: 4.936561822891235
StructuredData size:  13095
Processed:  45100
Elapsed time: 4.950614929199219
StructuredData size:  13127
Processed:  45200
Elapsed time: 4.962845802307129
StructuredData size:  13155
Processed:  45300
Elapsed time: 4.974162817001343
StructuredData size:  13183
Processed:  45400
Elapsed time: 4.983109951019287
StructuredData size:  13208
Processed:  45500
Elapsed time: 4.996261835098267
StructuredData size:  13234
Processed:  45600
Elapsed time: 5.007784843444824
StructuredData size:  13270
Processed:  45700
Elapsed time: 5.019876718521118
StructuredData

...

Processed:  574900
Elapsed time: 16785.088761091232
StructuredData size:  168821
Processed:  575000
Elapsed time: 16831.76729297638
StructuredData size:  168846
Processed:  575100
Elapsed time: 16854.77852010727
StructuredData size:  168879
Processed:  575200
Elapsed time: 16858.549292087555
StructuredData size:  168907
Processed:  575300
Elapsed time: 16873.891091823578
StructuredData size:  168943
Processed:  575400
Elapsed time: 16892.71687078476
StructuredData size:  168971
Processed:  575500
Elapsed time: 16904.64110684395
StructuredData size:  168990
Processed:  575600
Elapsed time: 16916.1938829422
StructuredData size:  169019
Processed:  575700
Elapsed time: 16964.43072795868
StructuredData size:  169050
Processed:  575800
Elapsed time: 16979.766307115555
StructuredData size:  169084
Processed:  575900
Elapsed time: 16995.492730140686
StructuredData size:  169115
Processed:  576000
Elapsed time: 17003.15751504898
StructuredData size:  169148
Processed:  576100
Elapsed time: 170

Processed:  585100
Elapsed time: 18811.996111869812
StructuredData size:  171736
Processed:  585200
Elapsed time: 18857.438363790512
StructuredData size:  171764
Processed:  585300
Elapsed time: 18880.107319116592
StructuredData size:  171794
Processed:  585400
Elapsed time: 18911.324843883514
StructuredData size:  171817
Processed:  585500
Elapsed time: 18930.463996887207
StructuredData size:  171849
Processed:  585600
Elapsed time: 18945.591490983963
StructuredData size:  171878
Processed:  585700
Elapsed time: 18957.00754094124
StructuredData size:  171901
Processed:  585800
Elapsed time: 18992.42369413376
StructuredData size:  171924
Processed:  585900
Elapsed time: 19011.50183200836
StructuredData size:  171959
Processed:  586000
Elapsed time: 19043.550264120102
StructuredData size:  171982
Processed:  586100
Elapsed time: 19063.50555586815
StructuredData size:  172016
Processed:  586200
Elapsed time: 19083.034563064575
StructuredData size:  172043
Processed:  586300
Elapsed time:

Processed:  595300
Elapsed time: 20964.615521907806
StructuredData size:  174710
Processed:  595400
Elapsed time: 21007.028263807297
StructuredData size:  174738
Processed:  595500
Elapsed time: 21022.450482845306
StructuredData size:  174760
Processed:  595600
Elapsed time: 21033.78006887436
StructuredData size:  174789
Processed:  595700
Elapsed time: 21045.80972290039
StructuredData size:  174818
Processed:  595800
Elapsed time: 21083.05597114563
StructuredData size:  174846
Processed:  595900
Elapsed time: 21094.527856826782
StructuredData size:  174876
Processed:  596000
Elapsed time: 21113.79831814766
StructuredData size:  174905
Processed:  596100
Elapsed time: 21152.75864291191
StructuredData size:  174932
Processed:  596200
Elapsed time: 21168.966735839844
StructuredData size:  174966
Processed:  596300
Elapsed time: 21205.488950014114
StructuredData size:  174993
Processed:  596400
Elapsed time: 21224.749211788177
StructuredData size:  175014
Processed:  596500
Elapsed time: 

Processed:  605500
Elapsed time: 23231.128572940826
StructuredData size:  177726
Processed:  605600
Elapsed time: 23242.402268886566
StructuredData size:  177761
Processed:  605700
Elapsed time: 23257.558471918106
StructuredData size:  177788
Processed:  605800
Elapsed time: 23280.415179014206
StructuredData size:  177809
Processed:  605900
Elapsed time: 23291.833442926407
StructuredData size:  177838
Processed:  606000
Elapsed time: 23299.449002981186
StructuredData size:  177875
Processed:  606100
Elapsed time: 23326.538400888443
StructuredData size:  177909
Processed:  606200
Elapsed time: 23361.35337805748
StructuredData size:  177940
Processed:  606300
Elapsed time: 23422.33323907852
StructuredData size:  177972
Processed:  606400
Elapsed time: 23429.98169183731
StructuredData size:  177998
Processed:  606500
Elapsed time: 23430.004793167114
StructuredData size:  178029
Processed:  606600
Elapsed time: 23441.198071956635
StructuredData size:  178057
Processed:  606700
Elapsed time

KeyboardInterrupt: 

In [31]:
print(stats)

{'tables': 144381, 'columns': 954112, 'synonyms': 0, 'relatedTo': 0, 'isA': 0, 'derivedFrom': 0, 'wikipediaResults': 0, 'lcsAmb': 0}


In [34]:
## caches
start_time = time.time()
saveCache('./cacheSynonym-small.json', cacheSynonym)
saveCache('./cacheRelatedTo-small.json', cacheRelatedTo)
saveCache('./cacheIsA-small.json', cacheIsA)
saveCache('./cacheDerivedFrom-small.json', cacheDerivedFrom)
saveCache('./cacheWikipedia-small.json', cacheWikipedia)
saveCache('./cacheAlias.json', cachedAlias)
total_time = time.time() - start_time
print("Process ended in time:", total_time)

Process ended in time: 129.26367211341858


In [35]:
a_file = open("cacheProvenance.pkl", "wb")
pickle.dump(cachedProvenance, a_file)
a_file.close()

In [25]:
def generateRow(schemaString, attr1, attr2, valueString):
    return schemaString + " attr1: " + attr1 + " attr2: " + attr2 + "\t" + valueString+"\n"

start_time = time.time()
examples = []
distinctLabels = set()
distinctForType = {}
for t in structuredData:
    table = t[0]
    columns = t[1]
    setPairs = t[2]
    ambiguousLabels = t[3]
    provenance = t[4]
    schemaString = "|".join(columns)
    #TODO: implement it
    #schemaString = generateFromData(table, columns)
    countGenerated = 0
    for attr1, attrSet in setPairs.items():
        for attr2 in attrSet:
            amb1 = ambiguousLabels[attr1]
            amb2 = ambiguousLabels[attr2]
            ambValues = set(amb1).intersection(set(amb2))
            ambValues.discard(attr1.lower())
            ambValues.discard(attr2.lower())
            prov1 = provenance[attr1]
            prov2= provenance[attr2]
            for ambValue in ambValues:
                distinctLabels.add(ambValue)
                for key, valueSet in prov1.items():
                    if ambValue in valueSet:
                        stats[key] = stats[key] + 1
                        if key not in distinctForType:
                            distinctForType[key] = set()
                        distLabelType = distinctForType[key]
                        distLabelType.add(ambValue)
                for key, valueSet in prov2.items():
                    if ambValue in valueSet:
                        stats[key] = stats[key] + 1
                        if key not in distinctForType:
                            distinctForType[key] = set()
                        distLabelType = distinctForType[key]
                        distLabelType.add(ambValue)
                example = generateRow(schemaString, attr1, attr2, ambValue)
                examples.append(example)
                countGenerated += 1
    for i in range(0, countGenerated):
        #TODO improve the column selection
        attr1 = choice(list(columns))
        attr2 = choice(list(columns))
        if (attr1 not in setPairs):
            example = generateRow(schemaString, attr1, attr2, "None")
            examples.append(example)
        else:
            attr1AmbSet = setPairs[attr1]
            if ((attr2 != attr1) and (attr2 not in attr1AmbSet)):
                example = generateRow(schemaString, attr1, attr2, "None")
                examples.append(example)
                
print(stats)
print("**** DISTINCT *****")
for key, value in distinctForType.items():
    print(key, len(value))
print("Unique labels: ", len(distinctLabels))
total_time = time.time() - start_time
print("Process ended in time:", total_time)

{'tables': 259583, 'columns': 1716076, 'synonyms': 275414, 'relatedTo': 994652, 'isA': 890928, 'derivedFrom': 28814, 'wikipediaResults': 573616, 'lcsAmb': 253564}
**** DISTINCT *****
wikipediaResults 2762
relatedTo 2077
isA 925
synonyms 1051
derivedFrom 386
lcsAmb 257
Unique labels:  6102
Process ended in time: 11.384925127029419


In [26]:
print("Examples:", len(examples))
examplesNoDuplicates = set(examples)
print("Examples no duplicates:", len(examplesNoDuplicates))
start_time = time.time()
fileName = "./generated/trainWDC-04-" + str(K) + "-v2.0.tsv"
#f = open("./trainWDC-00-5k-v1.1.tsv", "w")
f = open(fileName, "w")
#f.writelines(examples)
f.writelines(examplesNoDuplicates)
f.close()
total_time = time.time() - start_time
print("Process ended in time:", total_time)

Examples: 2117828
Examples no duplicates: 288085
Process ended in time: 0.2972440719604492
